### AWQ
[Activation-Aware Weight Quantization (AWQ)](https://github.com/mit-han-lab/llm-awq) 也是一種 Post Training Optimization 的量化方法，同樣也是透過少量的校準資料集來進行更精準的量化。
* 相較於 GPTQ 而言，作者宣稱 AWQ 更不容易發生校準資料集的過度擬合問題，而且 AWQ 量化的模型在使用時可以有更快的推論速度。
    * 作者指出，像是 GPTQ 這種透過**反向傳播 (Backward)** 來進行量化的方法，可能會使模型在校準資料集上發生過度擬合的情形。
    * 因此AWQ並不仰賴反向傳播，只透過**前向運算 (Forward)** 來觀察**輸出分佈 (Activation Distribution)** 並藉此尋找**重要的模型權重 (Salient Weights)**。
    * 作者指出，只要將其中 1% 的重要權重保留為 FP16 的資料型態，就能大幅降低量化帶來的誤差。
* 但如果為了保護這些分散在不同地方的重要權重，而使得整份權重量化完後是混精度的，則會使計算效率降低。
    * 因此作者引入 **Activation-Aware Scaling** 的技巧來對每組權重做量化，使得重要權重能夠獲得保護的同時，也能夠避免混精度權重以提昇硬體友善度，降低運算瓶頸，在實際推論上可以有相當快的速度。
* 在社群上也有類似 AutoGPTQ 的 [AutoAWQ](https://github.com/casper-hansen/AutoAWQ) 這種方便的工具可用，AWQ 量化的過程是一層一層的量化，所以 AutoAWQ 可以將模型權重本身放在 CPU 裡面，只有在該層需要量化時，才把它的權重拿出來放在 GPU 記憶體裡面，因此記憶體的使用非常有效率。

#### 1. 若要使用 AWQ 量化，首先需要安裝 AutoAWQ 套件：

In [ ]:
%%bash
pip install autoawq

#### 2. 可以使用 `AutoAWQForCausalLM` 類別讀取原始模型：
* 這裡將模型放在 CPU 記憶體裡面，避免模型權重佔用太多 GPU 記憶體導致 AutoAWQ 沒有空間進行量化運算。

In [ ]:
import torch

from awq import AutoAWQForCausalLM

model_dir = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoAWQForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.float16,
    device_map="cpu",   # 權重可以放在 CPU 就好
)

#### 3. 接下來會需要準備一份校準資料集
* 預設會使用原作者提供的 [`mit-han-lab/pile-val-backup` 這份資料集](https://huggingface.co/datasets/mit-han-lab/pile-val-backup) 進行校準。
* 但我們也可以自行準備一份字串列表當作資料集：
* 注意：
    * 如果給的文本太短太少的話， 進行量化時就會發生 `RuntimeError: torch.cat() expected a non-empty list of Tensors` 的錯誤。
    * 這是因為在套件裡面的 `awq/utils/calib_data.py`，作者放了一個 Token 數量必須大於 512 才會拿來用的條件，可能是因為[原本 AWQ 的實做](https://github.com/casper-hansen/AutoAWQ/issues/191)就是如此的關係。


In [ ]:
# 此為示範用的校準資料集，請根據自身應用替換成適當資料集
# 只要是 list[str] 的型態且 Token 數量夠多就能量化

calib_data = ["Hello, World! "  *  8] * 16

#### 4. 接下來放上相關設定，並開始進行量化：
* 筆者實測使用 RTX 3090 24 GB 量化一個 7B 模型，大約也是七八分鐘左右，速度上與 GPTQ 是差不多的，而且過程中所使用的 GPU 記憶體非常少量。 
* 如果 CPU 記憶體夠大，也能拿來量化 70B 的模型，大概要花費 100 分鐘左右。 
* 不過 70B 的模型即便量化到 4-Bit 也放不進一張 24GB GPU 裡面就是了。

In [ ]:
from transformers import AutoTokenizer as TkCls

tokenizer = TkCls.from_pretrained(model_dir)

# 相關設定
quant_config = dict(
    zero_point=True,
    q_group_size=128,
    w_bit=4,
    version="GEMM",
)

# 進行量化
model.quantize(
    tokenizer,
    quant_config=quant_config,
    calib_data=calib_data,
)

# 儲存量化結果
output_dir = "Models/Llama3-8B-Inst-AWQ"
model.save_quantized(output_dir)
tokenizer.save_pretrained(output_dir)

#### 5. 這些量化完的模型，可以直接讀取進來使用：
* 不需要任何額外的設定就能讀取，且 AWQ 模型在讀取速度上快滿多的。

In [ ]:
from transformers import LlamaForCausalLM as ModelCls

model_dir = "Models/Llama3-8B-Inst-AWQ"

model: ModelCls = ModelCls.from_pretrained(
    model_dir,
    device_map="auto",
)   # 約 5.8 GiB